### Automate Event Planning

In [4]:
# Warning control
import os
import warnings
from dotenv import load_dotenv,find_dotenv

warnings.filterwarnings('ignore')
load_dotenv(find_dotenv())

True

In [5]:
from phoenix.otel import register

# configure the Phoenix tracer
tracer_provider = register(
  project_name="my-crewai-vertex-app", # Default is 'default'
  auto_instrument=True # Auto-instrument your app based on installed OI dependencies
)

Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: my-crewai-vertex-app
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [17]:
# Run gcloud auth application-default login before running this script
# to set up the credentials for the Google Cloud SDK
# and the Google Cloud client libraries.
# SERPER API key is required for this example. 
# You can set it up in your environment variables or directly in the code.


from crewai import LLM
from google.auth import default

credentials, _ = default()

llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0.7,
    credentials=credentials,
    vertex_project=os.getenv("GOOGLE_CLOUD_PROJECT_ID"),
    vertex_location=os.getenv("GOOGLE_CLOUD_PROJECT_REGION"),
)

llm.call("Hello there Gemini! How are you doing today?")

"Hello! I'm doing well, thank you for asking. As a large language model, I don't experience emotions like humans do, but I'm functioning optimally and ready to assist you. How can I help you today?\n"

- Import libraries, APIs and LLM

In [18]:
from crewai import Agent, Crew, Task

## crewAI Tools

In [19]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [20]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm=llm,
)

In [12]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=llm
)

In [21]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=llm
)

## Creating Venue Pydantic Object

- Create a class `VenueDetails` using [Pydantic BaseModel](https://docs.pydantic.dev/latest/api/base_model/).
- Agents will populate this object with information about different venues by creating different instances of it.

In [22]:
from pydantic import BaseModel
# Define a Pydantic model for venue details 
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str
    best_suited_for: str
    indoor_or_outdoor: str

## Creating Tasks
- By using `output_json`, you can specify the structure of the output you want.
- By using `output_file`, you can get your output in a file.
- By setting `human_input=True`, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [23]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

- By setting `async_execution=True`, it means the task can run in parallel with the tasks which come after it.

In [27]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=False,
    agent=logistics_manager
)

In [28]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

## Creating the Crew

**Note**: Since you set `async_execution=True` for `logistics_task` and `marketing_task` tasks, now the order for them does not matter in the `tasks` list.

In [29]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [30]:
event_details = {
    'event_topic': "Chennai AI Team - Innovation Conference",
    'event_description': "A gathering of tech innovators, college students, industry professionals in AI and Data Science and top leaders in the AI space",
    'event_city': "Chennai, Tamil Nadu, OMR or ECR",
    'tentative_date': "2025-06-01",
    'expected_participants': 100,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2**: 
- Since you set `human_input=True` for some tasks, the execution will ask for your input before it finishes running.
- When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.

In [31]:
result = event_management_crew.kickoff(inputs=event_details)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: eb870044-d39f-4e74-868f-7d5088ec71ab                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 25e61c89-565b-4775-be1d-cc83b8de38e5
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 25e61c89-565b-4775-be1d-cc83b8de38e5
       Status: Executing Task...
    └── 🤖 Agent: Venue Coordinator
            Status: In Progress

# Agent: Venue Coordinator
## Task: Find a venue in Chennai, Tamil Nadu, OMR or ECR that meets criteria for Chennai AI Team - Innovation Conference.


🤖 Agent: Venue Coordinator
    Status: In Progress
└── 🧠 Thinking...

...



# Agent: Marketing and Communications Agent
## Final Answer: 
```markdown
# Chennai AI Team - Innovation Conference: Marketing and Attendee Engagement Plan

## Goal

Engage at least 100 potential attendees for the Chennai AI Team - Innovation Conference.

## Target Audience

*   AI researchers and academics
*   AI innovators and startups
*   Technology enthusiasts
*   Business professionals interested in AI applications
*   Students in relevant fields (Computer Science, Data Science, etc.)

## Marketing Activities

### 1. Pre-Conference Promotion (4-6 weeks prior)

*   **Website and Landing Page:**
    *   Create a dedicated conference website with detailed information: agenda, speakers, venue, registration details, and FAQs.
    *   Ensure the website is mobile-friendly and SEO-optimized.
    *   Implement a clear call-to-action (CTA) for registration.
*   **Social Media Marketing:**
    *   **Platforms:** LinkedIn, Twitter, Facebook, Instagram. (Consider local platforms popular in 

🚀 Crew: crew
├── 📋 Task: 25e61c89-565b-4775-be1d-cc83b8de38e5
│      Assigned to: Venue Coordinator
│      Status: ✅ Completed
│   └── 🤖 Agent: Venue Coordinator
│           Status: ✅ Completed
├── 📋 Task: 911700c2-75fd-499a-b982-bba0dad2f78b
│      Assigned to: Logistics Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Logistics Manager
│           Status: ✅ Completed
└── 📋 Task: 4afd94e5-f041-4399-a804-bf5cca6629db
       Status: Executing Task...
    └── 🤖 Agent: Marketing and Communications Agent
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 25e61c89-565b-4775-be1d-cc83b8de38e5
│      Assigned to: Venue Coordinator
│      Status: ✅ Completed
│   └── 🤖 Agent: Venue Coordinator
│           Status: ✅ Completed
├── 📋 Task: 911700c2-75fd-499a-b982-bba0dad2f78b
│      Assigned to: Logistics Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Logistics Manager
│           Status: ✅ Completed
└── 📋 Task: 4afd94e5-f041-4399-a804-bf5cca6629db
       Assigned to: Marketing and Communications Agent
       Status: ✅ Completed
    └── 🤖 Agent: Marketing and Communications Agent
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 4afd94e5-f041-4399-a804-bf5cca6629db                                                                     │
│  Agent: Marketing and Communications Agent                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: eb870044-d39f-4e74-868f-7d5088ec71ab                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

- Display the generated `venue_details.json` file.

In [32]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)

{'address': 'Olympia Technology Park, Chennai, Tamil Nadu, 600032, IND',
 'best_suited_for': 'Conferences, client meetings, training sessions, '
                    'interviews, collaborative work',
 'booking_status': 'Available for booking. Contact +91 22 6786 9561 to check '
                   'availability and book.',
 'capacity': 0,
 'indoor_or_outdoor': 'Indoor',
 'name': 'Regus - Altius Building, Olympia Technology Park'}


- Display the generated `marketing_report.md` file.

**Note**: After `kickoff` execution has successfully ran, wait an extra 45 seconds for the `marketing_report.md` file to be generated. If you try to run the code below before the file has been generated, your output would look like:

```
marketing_report.md
```

If you see this output, wait some more and than try again.

In [33]:
from IPython.display import Markdown
Markdown("marketing_report.md")

```markdown
# Chennai AI Team - Innovation Conference: Marketing and Attendee Engagement Plan

## Goal

Engage at least 100 potential attendees for the Chennai AI Team - Innovation Conference.

## Target Audience

*   AI researchers and academics
*   AI innovators and startups
*   Technology enthusiasts
*   Business professionals interested in AI applications
*   Students in relevant fields (Computer Science, Data Science, etc.)

## Marketing Activities

### 1. Pre-Conference Promotion (4-6 weeks prior)

*   **Website and Landing Page:**
    *   Create a dedicated conference website with detailed information: agenda, speakers, venue, registration details, and FAQs.
    *   Ensure the website is mobile-friendly and SEO-optimized.
    *   Implement a clear call-to-action (CTA) for registration.
*   **Social Media Marketing:**
    *   **Platforms:** LinkedIn, Twitter, Facebook, Instagram. (Consider local platforms popular in Chennai).
    *   **Content:**
        *   Engaging posts highlighting speakers, key sessions, and the value of attending.
        *   Share relevant AI news, articles, and research to attract a wider audience.
        *   Run polls and Q&A sessions related to AI topics.
        *   Use relevant hashtags (e.g., #ChennaiAI, #AIInnovation, #ArtificialIntelligence, #IndiaAI).
        *   Create visually appealing content: images, videos, and infographics.
    *   **Paid Advertising:**
        *   Targeted ads on LinkedIn and other platforms based on demographics, interests, and job titles.
        *   A/B test different ad creatives and targeting options to optimize performance.
*   **Email Marketing:**
    *   Build an email list through website sign-ups and targeted outreach.
    *   Send regular newsletters with conference updates, speaker spotlights, and early bird registration discounts.
    *   Segment the email list based on interests and send personalized messages.
*   **Public Relations and Media Outreach:**
    *   Send press releases to local media outlets and technology publications.
    *   Reach out to journalists and bloggers covering AI and technology in Chennai and India.
    *   Offer exclusive interviews with speakers or organizers.
*   **Partnerships and Sponsorships:**
    *   Collaborate with local universities, AI research institutions, and technology companies.
    *   Offer sponsorship packages with branding opportunities and speaking slots.
    *   Cross-promote the conference through partner channels.
*   **Content Marketing:**
    *   Create blog posts, articles, and white papers on relevant AI topics.
    *   Share these resources on the conference website and social media channels.
    *   Invite guest bloggers and industry experts to contribute content.
*   **Early Bird Discounts and Incentives:**
    *   Offer discounted registration fees for early sign-ups.
    *   Provide exclusive access to workshops or networking events for early registrants.

### 2. During Conference Engagement

*   **Mobile App:**
    *   Develop a conference mobile app with agenda, speaker bios, maps, networking features, and real-time updates.
    *   Use the app to facilitate Q&A sessions and polls during presentations.
*   **Networking Opportunities:**
    *   Organize dedicated networking breaks and social events.
    *   Facilitate introductions between attendees with similar interests.
    *   Use AI-powered matchmaking tools (if available) to connect attendees.
*   **Interactive Sessions:**
    *   Incorporate interactive elements into presentations, such as live demos, workshops, and group discussions.
    *   Use audience response systems (e.g., Mentimeter) to engage attendees and gather feedback.
*   **Social Media Engagement:**
    *   Encourage attendees to share their experiences on social media using the conference hashtag.
    *   Live-tweet key sessions and highlight interesting quotes.
    *   Run contests and giveaways to incentivize social media participation.
*   **Feedback Collection:**
    *   Collect feedback from attendees through surveys and feedback forms.
    *   Use the feedback to improve future conferences.

### 3. Post-Conference Follow-up

*   **Thank You Emails:**
    *   Send thank you emails to all attendees with links to presentation slides, recordings, and other resources.
*   **Post-Conference Survey:**
    *   Send a survey to gather feedback on the conference experience.
    *   Ask about the quality of the sessions, networking opportunities, and overall organization.
*   **Content Repurposing:**
    *   Repurpose conference content into blog posts, articles, and videos.
    *   Share these resources on the conference website and social media channels.
*   **Lead Nurturing:**
    *   Follow up with leads generated at the conference through targeted email campaigns.
    *   Offer exclusive content and resources to nurture leads.
*   **Community Building:**
    *   Create an online community (e.g., a LinkedIn group or forum) for attendees to connect and share ideas.
    *   Organize regular online events and webinars to keep the community engaged.

## Attendee Engagement Metrics

*   **Website Traffic:** Track website visits, page views, and bounce rate.
*   **Social Media Reach and Engagement:** Monitor likes, shares, comments, and hashtag usage.
*   **Email Open and Click-Through Rates:** Track email engagement metrics to optimize email campaigns.
*   **Registration Numbers:** Monitor the number of registered attendees and track registration trends.
*   **Conference App Usage:** Track app downloads, active users, and feature usage.
*   **Survey Response Rates:** Monitor the number of survey responses and analyze the feedback.
*   **Lead Generation:** Track the number of leads generated at the conference.
*   **Post-Conference Engagement:** Monitor website traffic, social media engagement, and email engagement after the conference.

## Marketing Budget

*   Allocate budget for website development, social media advertising, email marketing, public relations, content creation, and event management.

## Team Roles and Responsibilities

*   Assign specific roles and responsibilities to team members for each marketing activity.
*   Establish clear communication channels and reporting mechanisms.

## Tools and Technologies

*   Website platform (e.g., WordPress)
*   Social media management tools (e.g., Hootsuite, Buffer)
*   Email marketing platform (e.g., Mailchimp, ConvertKit)
*   CRM system (e.g., Salesforce, HubSpot)
*   Conference mobile app platform
*   Survey platform (e.g., SurveyMonkey, Google Forms)
*   Analytics tools (e.g., Google Analytics)

## Success Measurement

*   Achieve the goal of engaging at least 100 potential attendees.
*   Generate a positive return on investment (ROI) from the marketing activities.
*   Increase brand awareness and establish the Chennai AI Team as a leader in the AI community.
*   Gather valuable feedback from attendees to improve future conferences.

This plan provides a comprehensive framework for marketing the Chennai AI Team - Innovation Conference and engaging potential attendees. By implementing these strategies and tracking the results, the Chennai AI Team can achieve its goals and create a successful and impactful event.
```